# 🏗 Groundwork 🪚

In [17]:
from glob import glob
from os.path import isfile
from os import system, remove
from openpyxl import load_workbook
from pandas import ExcelWriter, DataFrame, ExcelFile, merge, concat, read_excel

xlsx = 'unite.xlsx'
chief = 'chief'
payment = 'payment'
packet = 'packet'
edrpou = 'edrpou'
months = 10
packet_name = 'packet_name'
medinst = 'Назва закладу'
regarea = 'Область реєстрації'
ownership ='Форма власності'
monthly_finance = ['M1','M2','M3','M4','M5','M6','M7','M8','M9','M10']

# 👷🏾‍♂️ Functional 👨🏽‍🏭

In [18]:
def newfile(name:str): 
    if isfile(xlsx): remove(xlsx)
    ExcelWriter(name).save()

def capacity(typefile:str)->list: return sorted([ file for file in glob(f'*.{typefile}') if '~$' not in file ])

def fill(file:str, namesheet:str, content:list, columns:list=None, mode:str='w', index:bool=False, header:bool=True):
    with ExcelWriter(file, mode=mode, engine="openpyxl", if_sheet_exists=(None if mode=='w' else 'overlay')) as writer:  
        content.to_excel(writer, sheet_name=namesheet, index=index, header=header, columns=columns)
        
def unitexls(fxls:list):
    list_sheet=[]
    for fxl in fxls:
        ef = ExcelFile(fxl)
        print('\n', fxl, ":", end=' ')
        for sheet in ef.sheet_names:
            list_sheet.append(sheet) if sheet not in list_sheet else list_sheet.append(sheet+'_'+fxl.replace('.xlsx',''))
            fill(content=read_excel(ef, sheet_name=sheet), file=xlsx, namesheet=list_sheet[-1], mode='a', header=True)
            print(list_sheet[-1], end=' ')

def rename(file:str, sheet:str, oldnamesheet:str='Sheet1'):
    fxl = load_workbook(file)
    fxl[oldnamesheet].title = sheet
    fxl.save(file)

def check(file:str, message:str='\nResult of checking:'):
    fxl = load_workbook(file)
    sheets = fxl.sheetnames
    print(message, sheets, end='\n\n')

def scan(file:str, namesheet:str, start:str='A', upto:str='A1') -> DataFrame:
    tabl = []
    fxl = load_workbook(file)
    cells = fxl[namesheet][start+'1':upto]
    columns = [ cells[0][c].value for c in range(len(cells[0])) ]
    for cell in cells[1:]: tabl.append([ cell[c].value for c in range(len(cell)) ])
    return DataFrame( tabl, columns=columns, index=list(range(1,int(upto[1:]))) )

def connect(dfs:DataFrame) -> DataFrame:
    for df in dfs[1:]: dfs[0] = dfs[0].join(df,how='right')
    return dfs[0]

def column2rows(items:int, df:DataFrame) -> list:
    L={}
    D=[]
    for enum in range(1,items):
        for k,v in df.values:
            if int(k)==enum:
                D.append(v)
        if D!=[]: 
            L[enum]=D
            D=[]
    L = list(L.values())
    return L

def enable(file:str, turnoff:str="Microsoft Excel", afterseconds:int=60): system(f'''open {file}; sleep {afterseconds};   killall "{turnoff}" || taskkill /f /im {turnoff.split()[-1]}.exe /t''') # mac || win

# 🫡 Screenplay 📝

In [20]:
newfile(xlsx)
fxls = capacity('xlsx')
fxls.remove(xlsx)
unitexls(fxls)
rename(xlsx, chief)
check(xlsx)

# we work in file payment
dfme = scan(file=xlsx, namesheet=payment, upto='A87')
dfmp = scan(file=xlsx, namesheet=payment, upto='B87', start='B')
df = concat([ connect( [dfme,dfmp] ) ], ignore_index=True).drop_duplicates(ignore_index=True)

# we work in file packet
dfp = scan(file=xlsx, namesheet=packet, upto='B46')
dfp = DataFrame([ nm for p in df['packet'] for id,nm in dict(dfp.values).items() if p==id ], columns=['packet_name'])
df = connect( [df,dfp] )

# we work in file edrpou
dfef = scan(file=xlsx, namesheet=edrpou, upto='B4961')
dfee = scan(file=xlsx, namesheet=edrpou, upto='C4961', start='C')
dfee.rename(columns={'ЄДРПОУ':'edrpou'}, inplace = True)
dfen = scan(file=xlsx, namesheet=edrpou, upto='D4961', start='D')
dfef = connect( [dfee,dfen,dfef] )
df = merge(df,dfef,on='edrpou').drop_duplicates(subset=['edrpou','packet'],ignore_index=True)

# add cash costs from payment file
dfmm = scan(file=xlsx, namesheet=payment, upto='D87', start='C')
dfmm = DataFrame( column2rows(months,dfmm), columns=monthly_finance )
df = connect( [df,dfmm] )

# write everything in xlsx file and rejoice
print(df, end='\n\n')
fill(xlsx, chief, df, mode='a')
enable(xlsx)


Result of checking: ['chief', 'edrpou', 'packet', 'payment']

    edrpou  packet                                        packet_name  \
0   184945       3  Хірургічні операції дорослим та дітям у стаціо...   
1   184945       4  Стаціонарна допомога дорослим та дітям без про...   
2   185011       3  Хірургічні операції дорослим та дітям у стаціо...   
3   185011       4  Стаціонарна допомога дорослим та дітям без про...   
4  1110854       3  Хірургічні операції дорослим та дітям у стаціо...   
5  1110854       4  Стаціонарна допомога дорослим та дітям без про...   
6  1111032       3  Хірургічні операції дорослим та дітям у стаціо...   
7  1111032       4  Стаціонарна допомога дорослим та дітям без про...   
8  1111581       3  Хірургічні операції дорослим та дітям у стаціо...   

                                       Назва закладу Область реєстрації  \
0  КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО "КИЇВСЬКА...             М.КИЇВ   
1  КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО "КИЇВСЬКА...    